In [4]:
!pip3 install imblearn

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
     l     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0.0/258.3 kB ? eta -:--:--━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╸━━━ 235.5/258.3 kB 7.2 MB/s eta 0:00:01━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.3/258.3 kB 6.7 MB/s eta 0:00:00


In [5]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt


from sklearn.model_selection import train_test_split
import pickle
from tqdm.notebook import tqdm 
import pyarrow as pa
import pyarrow.parquet as pq
import torch
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score
import seaborn as sns
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset

import sys
sys.path.append('..')
from utils.preprocess import NDF
from collections import Counter

from imblearn.over_sampling import SMOTE
from sklearn import metrics

In [ ]:
from utils.preprocess import NDF

input_data = {
    'benign': '../feature-extraction/floor/benign_combined.parquet',
    #'malign': 'parkets/malware_bp.parquet'
    #'malign': 'parkets/phishing_final_2024.parquet'
    'malign': '../feature-extraction/floor/phishing_4.parquet'
}
dataset = NDF("cnn", True, input_data=input_data, one_line_processing=False)

2024-06-12 20:10:56,965 - utils.preprocess - INFO - Benign dataset path: ../feature-extraction/floor/benign_combined.parquet
2024-06-12 20:10:56,967 - utils.preprocess - INFO - Malign dataset path: ../feature-extraction/floor/phishing_4.parquet
2024-06-12 20:10:57,889 - utils.preprocess - INFO - Number of records in combined dataset: 1016263


In [2]:
# print total number of samples
print(f"Total number of samples: {len(dataset)}")

x_train, x_test, y_train, y_test = train_test_split(np.array(dataset['features']), np.array(dataset['labels']), test_size=0.05, random_state=42)


new_label_counts = Counter(y_train)
print("Class distribution:")
for label, count in new_label_counts.items():
    print(f"Class {label}: {count}")



# Apply SMOTE
smote = SMOTE(random_state=42)
x_train_smote, y_train_smote = smote.fit_resample(x_train, y_train)

# Check the new class distribution
new_label_counts = Counter(y_train_smote)
print("New class distribution after SMOTE:")
for label, count in new_label_counts.items():
    print(f"Class {label}: {count}")

# Convert the data to PyTorch tensors
x_train = torch.tensor(x_train_smote, dtype=torch.float32)
x_test = torch.tensor(x_test, dtype=torch.float32)  # Convert x_test to tensor
y_train = torch.tensor(y_train_smote, dtype=torch.long)
y_test = torch.tensor(y_test, dtype=torch.long) 
feature_size = x_train.shape[1]


NameError: name 'dataset' is not defined

In [6]:
import torch.nn.functional as F
import math

# Function to calculate the next perfect square greater than a given number
def next_perfect_square(n):
    next_square = math.ceil(n**0.5)**2
    return next_square

feature_size = x_train.shape[1]
desired_size = next_perfect_square(feature_size)
side_size = int(desired_size ** 0.5)

# Calculate padding required to achieve the desired size
padding = desired_size - feature_size

# Applying dynamic padding
if padding > 0:
    x_train_padded = F.pad(x_train, (0, padding), 'constant', 0)
    x_test_padded = F.pad(x_test, (0, padding), 'constant', 0)
else:
    x_train_padded = x_train
    x_test_padded = x_test

# Reshape the data to the new dynamically calculated square shape
x_train = x_train_padded.view(-1, 1, side_size, side_size)
x_test = x_test_padded.view(-1, 1, side_size, side_size)

In [10]:
class Net(nn.Module):
    def __init__(self, side_size):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, stride=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, stride=1)
        self.bn2 = nn.BatchNorm2d(64)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, stride=1)
        self.bn3 = nn.BatchNorm2d(128)
        
        # Adjust the size calculation based on the number of convolutional layers
        self.fc1 = nn.Linear(128 * (side_size-6)**2, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 128)
        self.fc4 = nn.Linear(128, 2)
        
        # Optionally use dropout
        self.dropout1 = nn.Dropout(0.2)
        self.dropout2 = nn.Dropout(0.2)
        
        print("CNN model created")

    def forward(self, x):
        x = F.relu(self.bn1(self.conv1(x)))
        x = F.relu(self.bn2(self.conv2(x)))
        x = F.relu(self.bn3(self.conv3(x)))
        
        x = torch.flatten(x, 1)
        x = self.dropout1(x)  # Dropout applied after flattening
        
        x = F.relu(self.fc1(x))
        x = self.dropout2(x)  # Dropout applied after first fully connected layer
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        return self.fc4(x) # No softmax here, as CrossEntropyLoss applies LogSoftmax internally

In [8]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [12]:
LEARNING_RATE = 0.000238
BATCH_SIZE = 64
EPOCHS = 15


model = Net(side_size).to(device)  # Classic fully connected model


#CANNOT RETURN SOFT MAX AS THE LOSS FUNCTION IS CROSS ENTROPY
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

# Prepare DataLoader
train_data = TensorDataset(x_train, y_train.long())
train_loader = DataLoader(train_data, batch_size=BATCH_SIZE, shuffle=True)

# Function to compute accuracy and F1 score
def compute_metrics(data_loader, model):
    model.eval()
    true_labels = []
    predictions = []

    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)  # Move data to the device
            output = model(data)
            pred = output.argmax(dim=1, keepdim=True)
            true_labels.extend(target.cpu().tolist())  # Move data back to CPU for scoring
            predictions.extend(pred.view_as(target).cpu().tolist())  # Move data back to CPU

    accuracy = accuracy_score(true_labels, predictions)
    f1 = f1_score(true_labels, predictions)
    return accuracy, f1

epoch_losses = []
epoch_accuracies = []
epoch_f1s = []

# start timer
import time


# Training loop
for epoch in range(EPOCHS):
    timer = time.time()
    model.train()
    running_loss = 0.0
    for data, target in train_loader:
        data, target = data.to(device), target.to(device)  # Move data to the device
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()

    # Compute average loss
    avg_loss = running_loss / len(train_loader)
    epoch_losses.append(avg_loss)

    # Evaluate model and store metrics
    train_accuracy, train_f1 = compute_metrics(train_loader, model)
    epoch_accuracies.append(train_accuracy)
    epoch_f1s.append(train_f1)

    # Enhanced logging
    print(f"Epoch {epoch+1}/{EPOCHS} - Loss: {avg_loss:.4f}, Accuracy: {train_accuracy:.4f}, F1 Score: {train_f1:.4f} Time taken: {round(time.time() - timer, 2)}s")



# Save the model
torch.save(model.state_dict(), './models/phishing_with_final_parquet.pth')
print("Model training complete and saved.")

CNN model created


KeyboardInterrupt: 

In [9]:
import torch
from torch.utils.data import DataLoader, TensorDataset
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, confusion_matrix
import numpy as np
from sklearn.metrics import roc_auc_score

def compute_metrics(data_loader, model, criterion):
    """
    Computes a wide range of performance metrics for the given model evaluated on the provided DataLoader.
    
    Parameters:
        data_loader (DataLoader): DataLoader for evaluation data.
        model (torch.nn.Module): The neural network model to evaluate.
        criterion (torch.nn.Module): Loss function used for the model.
    
    Returns:
        dict: A dictionary containing various performance metrics.
    """
    model.eval()  # Switch model to evaluation mode
    true_labels, predictions, probs, losses = [], [], [], []

    with torch.no_grad():
        for data, target in data_loader:
            output = model(data)
            loss = criterion(output, target)
            pred = output.argmax(dim=1, keepdim=True)
            prob = torch.nn.functional.softmax(output, dim=1)[:,1]  # Probability for class 1
            true_labels.extend(target.cpu().numpy())
            predictions.extend(pred.cpu().view_as(target).numpy())
            probs.extend(prob.cpu().numpy())
            losses.append(loss.item())
    
    # Compute metrics
    metrics = {
        'accuracy': accuracy_score(true_labels, predictions),
        'f1': f1_score(true_labels, predictions, average='weighted'),
        'precision': precision_score(true_labels, predictions, average='weighted', zero_division=0),
        'recall': recall_score(true_labels, predictions, average='weighted', zero_division=0),
        'roc_auc_score': roc_auc_score(true_labels, probs) if len(np.unique(true_labels)) > 1 else 0,
        'cross_entropy_loss': np.mean(losses),
    }
    
    return metrics

def evaluate_model(model, x_test, y_test, batch_size, criterion):
    """
    Evaluates the model on the test dataset and prints out a comprehensive set of performance metrics.
    
    Parameters:
        model (torch.nn.Module): The neural network model to evaluate.
        x_test (Tensor): Test dataset features.
        y_test (Tensor): Test dataset labels.
        batch_size (int): Batch size for data loading.
        criterion (torch.nn.Module): Loss function used for the model.
    """
    test_data = TensorDataset(x_test, y_test.long())
    test_loader = DataLoader(test_data, batch_size=batch_size)
    
    metrics = compute_metrics(test_loader, model, criterion)
    
    # Display the metrics
    print("Test Metrics:")
    for metric_name, metric_value in metrics.items():
        if metric_name == 'confusion_matrix':
            print(f"{metric_name}:\n{metric_value}\n")
        else:
            print(f"{metric_name}: {metric_value:.4f}")
    
    return metrics

# Assuming criterion is defined (e.g., nn.CrossEntropyLoss())
criterion = torch.nn.CrossEntropyLoss()

# Example of how to use the evaluate_model function:
# model = Net()  # Assume Net is defined elsewhere and is your trained model
x_test, y_test = x_test.to(device), y_test.to(device)  # Move data to the device
metrics = evaluate_model(model, x_test, y_test, BATCH_SIZE, criterion)


Test Metrics:
accuracy: 0.9902
f1: 0.9902
precision: 0.9902
recall: 0.9902
roc_auc_score: 0.9964
cross_entropy_loss: 0.0475
